<div style="background-color:dodgerblue;height:100px;"><br>
<center><h1><font style="color:white">The Path to Self-Discovery: Find Habits</font></h1></center>
</div>

# Deja Vu

Let's bring our data back!  Before, we loaded our DataFrame from a list of dictionaries.  Now, let's load it from a csv!

In [ ]:
import pandas as pd
df = pd.read_csv('./MyStructuredSearch.csv')

## Exercise 1

Remember when we cast our Lat, Lon, and Datestamp columns to the right datatypes?  Let's do it again!

# The Apply Function

Pandas has a wonderful transform function called the apply button.  It allows us to perform an operation against an entire Series, DataFrame, or Panel.

To show us how great it is, let's grab the weekday of each of our dates, so that we can more easily specify and filter by day.

First step: make a function that, given a datetime, will return its weekday. Note, 0 is Monday in Python land.

In [ ]:
def get_weekday(val):
    if not pd.isnull(val):
        return val.weekday()
    else:
        return None

Now, we can apply that to our Datestamp column, and make a new column!

In [ ]:
df['Weekday'] = df['Datestamp'].apply(get_weekday)

Let's better understand what weekdays have the most searches.

In [ ]:
df['Weekday'].value_counts()

And we can plot this to better understand what our data is telling us!

We won't go too detailed into plotting, but we can very easily make some histograms with pandas.

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%pylab inline

In [ ]:
df['Weekday'].plot('hist')

It looks like Mondays and Wednesdays, famously the least enjoyable days of the week, are also the days most searches are done.

## Exercise 7

What are the top days of the week during which the user searched for directions.
* Make a function that determines whether or not the search is for directions
* Filter for all rows where directions were searched for
* Plot the days of the week and number of times directions were searched each day

## Exercise 8

What are the top hours of the day during which the user is on his/her phone?

## Exercise 9

Create a column that identifies the website for each Visited activity.

`https://productforums.google.com/forum/` 

would be parsed as 

`productforums.google.com`

Then find the most visited site.

# Location, Location

As our last little neat skill, let's learn about distance!  First, let's calculate the centroid of the points.

In [ ]:
avg_lat = df['Lat'].mean(skipna=True)
avg_lon = df['Lon'].mean(skipna=True)
avg_loc = (avg_lat, avg_lon)

Python's Anaconda distribution has a great library called geopy that we can use to calculate the distance between two points!  For example, let's take the two ends of I-95: Miami, FL and Houlton, ME.

We can use the Vincenty method.  According to the great Wikipedia: "Vincenty's formulae are two related iterative methods used in geodesy to calculate the distance between two points on the surface of a spheroid, developed by Thaddeus Vincenty (1975a)."

In [ ]:
from geopy.distance import vincenty

miami = (25.7617, 80.1918)
houlton = (46.135256, 67.781246)

dist = vincenty(miami, houlton)
print(dist.miles)
print(dist.meters)

Now let's make a function that calculates a coordinates distance from the centroid!

In [ ]:
def dist_from_centroid(ser):
    if not pd.isnull(ser['Lat']):
        lat = ser['Lat']
        lon = ser['Lon']
        coord = (lat, lon)
        return vincenty(coord, avg_loc).miles
    return None

Now we want to use the apply function, but we want to use it across a set of multiple columns.  Fortunately, we can call apply on a whole DataFrame, not just a column!  We just have to tell pandas which axis we want to use, since by default, it will move along the 0th (or columnar) axis, rather than the rows (axis 1).

In [ ]:
df['Distance From Centroid'] = df.apply(dist_from_centroid, axis=1)

## Exercise 10
What was the location furthest from the centroid?  What were the searches made at this location?

The last piece of pandas we are going to explore is the `groupby` function.  This function allows us to group the data by one or more columns.  Basically, it creates little DataFrames associated with each of the groups.  Then we can perform operations on each of those and zip the results back up to each group!  

In [ ]:
df.groupby(['Lat', 'Lon']).size().head()

To make this more useful, we can sort the data as well (which we can do for Series, DataFrames, etc).

In [ ]:
df.groupby(['Lat', 'Lon']).size().sort_values(ascending=False).head()

# Dig Deep

Using what you have learned, dig into the data and find out some cool stuff!

Share what you learned here!

https://docs.google.com/forms/d/e/1FAIpQLSexLLJ3N64y7GnyXnLcbJxMxPiJOErBjtPorwb4uw8Q7novlw/viewform?usp=sf_link